In [1]:
import lightgbm as lgb
__file__=''
import os,sys
import pandas as pd
sys.path.append(os.path.join(os.path.dirname(__file__),'../LIB/'))
sys.path.append(os.path.join(os.path.dirname(__file__),'../../../../automl/automl_libs/'))
from env import FILE
from sklearn.feature_extraction import FeatureHasher
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
from tqdm import tqdm
from scipy import sparse 
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import chi2, SelectPercentile,SelectKBest
import gc
import time
from sklearn.feature_selection import chi2, SelectPercentile

In [2]:
from scipy.sparse import csr_matrix
def save_csr(matrix,file_prefix='csr'):
    data_file = file_prefix + '_data.npy'
    indices_file = file_prefix + '_indices.npy'
    indptr_file = file_prefix + '_indptr.npy'
    np.save(data_file,matrix.data)
    print('data save')
    np.save(indices_file,matrix.indices)
    print('indices save')
    np.save(indptr_file,matrix.indptr)
    print('indptr save')
    print('save done!')
    
def load_csr(file_prefix='csr'):
    data_file = file_prefix + '_data.npy'
    indices_file = file_prefix + '_indices.npy'
    indptr_file = file_prefix + '_indptr.npy'
    data = np.load(data_file)
    print('data load')
    indices = np.load(indices_file)
    print('indices load')
    indptr = np.load(indptr_file)
    print('indptr load')
    csr = csr_matrix((data, indices, indptr))
    return csr

In [3]:
train_csr_f = load_csr('../../data/features/xgb/train_csr_wfeats')
predict_csr_f = load_csr('../../data/features/xgb/predict_csr_wfeats')
train_y = np.load('../../data/features/xgb/train_y.npy')
predicted_file = pd.read_pickle('../../data/features/xgb/predict_id.pkl')

data load
indices load
indptr load
data load
indices load
indptr load


# Load More features

In [4]:
# X_fe = pd.read_pickle('../../data/features/xgb/new_feats_838.pkl')
# print(X_fe.shape)
# X_ = X_fe.copy()



In [5]:
# cut_bin = 30
# for col in tqdm(X_fe.columns):
#         X_[col] = pd.cut(X_fe[col], bins=cut_bin, labels=False, retbins=True, right=False)[0]  

# Append more data from SVD

In [6]:
# train_csr_f = load_csr('../../data/features/xgb/train_csr_2500')
# predict_csr_f = load_csr('../../data/features/xgb/predict_csr_2500')
# train_instance = pd.read_pickle('../../data/features/xgb/train_noclean_bytime.pkl')[['instance_id']].copy()
# train_y = np.load('../../data/features/xgb/train_y_f620_b70_col17k.npy')

# X_u = pd.read_pickle('../../data/features/agg/svd_utag_adid_cnt.pkl')
# X_u = train_instance.merge(X_u,on='instance_id',how='inner')
# X_u.drop('instance_id',inplace=True,axis=1)

# cut_bin = 30
# # selected_col = [col for col in X_u.columns if 'utagsIndividual_svd' not in col]
# # X_u = X_u[selected_col]
# for col in tqdm(X_u.columns):
#         X_u[col] = pd.cut(X_u[col], bins=cut_bin, labels=False, retbins=True, right=False)[0]  
        
# print(train_csr_f.shape)
# train_csr_f = sparse.hstack(
#     (sparse.csr_matrix(X_u.iloc[:train_csr_f.shape[0]]), train_csr_f), 'csr').astype(
#     'float32')
# predict_csr_f = sparse.hstack(
#     (sparse.csr_matrix(X_u.iloc[train_csr_f.shape[0]:]), predict_csr_f), 'csr').astype('float32')
# print(train_csr_f.shape)
        
        

In [7]:
# df_fea = pd.read_csv('../../data/features/xgb/fea_importance.csv')

In [8]:
# feature_select = SelectKBest(chi2, k=10000)
# feature_select.fit(train_csr, train_y)
# train_csr_10k = feature_select.transform(train_csr)
# predict_csr_10k = feature_select.transform(predict_csr)
# train_csr_10k.shape

In [9]:
# # train_csr_f = sparse.hstack(
# #     (sparse.csr_matrix(X_.iloc[:train_csr.shape[0]]), train_csr_10k), 'csr').astype(
# #     'float32')
# # predict_csr_f = sparse.hstack(
# #     (sparse.csr_matrix(X_.iloc[train_csr.shape[0]:]), predict_csr_10k), 'csr').astype('float32')



# train_csr_f = sparse.hstack(
#     (sparse.csr_matrix(X_.iloc[:-80276]), train_csr), 'csr').astype(
#     'float32')
# predict_csr_f = sparse.hstack(
#     (sparse.csr_matrix(X_.iloc[-80276:]), predict_csr), 'csr').astype('float32')

# print(train_csr_f.shape)
# print(predict_csr_f.shape)



In [10]:
# feature_select = SelectKBest(chi2, percentile=10000)
# feature_select.fit(train_csr, train_y)
# train_csr = feature_select.transform(train_csr)
# predict_csr = feature_select.transform(predict_csr)
# print('feature select')
# print(train_csr.shape)

# Generate sample

In [11]:
# sample_size = 400000
# sample_times = 2
# sample_all = pd.DataFrame({'data':range(train_csr.shape[0])})
# sample_index_list = []
# samples = []
# samples_y = []
# for i in tqdm(range(sample_times)):
#     sample_index_list.append(list(sample_all.sample(sample_size).index))
#     samples.append(train_csr[sample_index_list[i]])
#     samples_y.append(train_y[sample_index_list[i]])


# Define lgb model

In [12]:
lgb_model = lgb.LGBMClassifier(
    boosting_type='gbdt', num_leaves=32, reg_alpha=0, reg_lambda=0.1,
    max_depth=-1, n_estimators=6000, objective='binary',
    subsample=0.7, colsample_bytree=0.7, subsample_freq=1,
    learning_rate=0.05, random_state=1001, n_jobs=16
)


In [13]:
def test_on_sample(train_csr_f,train_y_f,lgb_model,n_fold = 5,seed=None,first=2500):
    if seed is None:
        seed = np.random.randint(1000)
    cv_list = []
    skf = StratifiedKFold(n_splits=n_fold, random_state=seed, shuffle=True)
    best_score = []


    feature_importance_tmp_split = []
    feature_importance_tmp_gain = []
    df_fea = pd.DataFrame()

    t1 = time.time()
    for index, (train_index, test_index) in enumerate(skf.split(train_csr_f, train_y_f)):
        lgb_model.fit(train_csr_f[train_index], train_y_f[train_index],
                      eval_set=[(train_csr_f[train_index], train_y_f[train_index]),
                                (train_csr_f[test_index], train_y_f[test_index])], early_stopping_rounds=200,verbose=200)
        best_score.append(lgb_model.best_score_['valid_1']['binary_logloss'])
        print(best_score)
        print('cv:',np.mean(best_score))
        test_pred = lgb_model.predict_proba(predict_csr_f, ntree_limit=lgb_model.best_iteration_)[:, 1]
        print('test mean:', test_pred.mean())
        feature_importance_tmp_split.append(lgb_model.booster_.feature_importance(importance_type='split'))
        feature_importance_tmp_gain.append(lgb_model.booster_.feature_importance(importance_type='gain'))

    df_fea['col_name'] = lgb_model.booster_.feature_name()
    df_fea['split'] = np.mean(feature_importance_tmp_split,axis=0)
    df_fea['gain'] = np.mean(feature_importance_tmp_gain,axis=0)

    cv_list.append(np.mean(best_score))
    t2 = time.time()
    print('time spend: {} s'.format(t2-t1))
    print('after dropping...')


    split_index = list(df_fea[df_fea.split!=0].index)
    gain_index = list(df_fea[df_fea.gain!=0].index)
    #union
#     retain_index = list(set(split_index+gain_index))
    #intersection
#     retain_index = list(set(split_index).intersection(set(gain_index)))
    if first is None:
        first = 500
    split_index = df_fea.sort_values('split',ascending=False)['split'].index.values[:first]
    gain_index = df_fea.sort_values('gain',ascending=False)['gain'].index.values[:first]
    retain_index = list(set(split_index).intersection(set(gain_index)))
    print(len(retain_index))

    best_scor2 = []
    t1 = time.time()
    for index, (train_index, test_index) in enumerate(skf.split(train_csr_f, train_y_f)):
        lgb_model.fit(train_csr_f[train_index][:,retain_index], train_y_f[train_index],
                      eval_set=[(train_csr_f[train_index][:,retain_index], train_y_f[train_index]),
                                (train_csr_f[test_index][:,retain_index], train_y_f[test_index])], early_stopping_rounds=200,verbose=200)
        best_scor2.append(lgb_model.best_score_['valid_1']['binary_logloss'])
        print(best_scor2)
        print('cv:',np.mean(best_scor2))
        #####################
        test_pred = lgb_model.predict_proba(predict_csr_f[:,retain_index], num_iteration=lgb_model.best_iteration_)[:, 1]
        print('test mean:', test_pred.mean())
        #####################
    cv_list.append(np.mean(best_scor2))
    drop = False
    if cv_list[-1] < cv_list[-2]:
        print('dropping successfully!')
        print('cv drop:{}'.format(cv_list[-2]-cv_list[-1]))
        drop = True
    else:
        print('dropping failed!!!!!!')
        print('cv increase:{}'.format(cv_list[-1]-cv_list[-2]))
    t2 = time.time()
    print('time spend: {} s'.format(t2-t1))
    return df_fea, retain_index, drop

In [14]:
def verify_on_sample(train_csr_f,train_y_f,lgb_model,retain_index,n_fold = 3,seed=None,cv_list=[]):
    if seed is None:
        seed = np.random.randint(1000)

    skf = StratifiedKFold(n_splits=n_fold, random_state=seed, shuffle=True)
    best_score = []
    
    if len(cv_list) == 0:

        t1 = time.time()
        for index, (train_index, test_index) in enumerate(skf.split(train_csr_f, train_y_f)):
            lgb_model.fit(train_csr_f[train_index], train_y_f[train_index],
                          eval_set=[(train_csr_f[train_index], train_y_f[train_index]),
                                    (train_csr_f[test_index], train_y_f[test_index])], early_stopping_rounds=100,verbose=200)
            best_score.append(lgb_model.best_score_['valid_1']['binary_logloss'])
            print(best_score)
            print('cv:',np.mean(best_score))




        cv_list.append(np.mean(best_score))
        t2 = time.time()
        print('time spend: {} s'.format(t2-t1))
        print('after dropping...')

        print(len(retain_index))

    best_scor2 = []
    t1 = time.time()
    for index, (train_index, test_index) in enumerate(skf.split(train_csr_f, train_y_f)):
        lgb_model.fit(train_csr_f[train_index][:,retain_index], train_y_f[train_index],
                      eval_set=[(train_csr_f[train_index][:,retain_index], train_y_f[train_index]),
                                (train_csr_f[test_index][:,retain_index], train_y_f[test_index])], early_stopping_rounds=100,verbose=200)
        best_scor2.append(lgb_model.best_score_['valid_1']['binary_logloss'])
        print(best_scor2)
        print('cv:',np.mean(best_scor2))
    cv_list.append(np.mean(best_scor2))
    t2 = time.time()
    print('time spend: {} s'.format(t2-t1))

    if cv_list[-1] < cv_list[-2]:
        print('dropping successfully!')
        print('cv drop:{}'.format(cv_list[-2]-cv_list[-1]))
        return True
    else:
        print('dropping failed!!!!!!')
        print('cv increase:{}'.format(cv_list[-1]-cv_list[-2]))
        return False
    
    

In [15]:
gc.collect()
# X_.iloc[:train_csr.shape[0]]
df_fea, retain_index, ifdrop = test_on_sample(train_csr_f,train_y,lgb_model,n_fold = 5,seed=41,first=2500)

Training until validation scores don't improve for 200 rounds.
[200]	valid_0's binary_logloss: 0.415873	valid_1's binary_logloss: 0.416471
[400]	valid_0's binary_logloss: 0.413655	valid_1's binary_logloss: 0.415363
[600]	valid_0's binary_logloss: 0.412072	valid_1's binary_logloss: 0.414986
[800]	valid_0's binary_logloss: 0.410689	valid_1's binary_logloss: 0.414795
[1000]	valid_0's binary_logloss: 0.409408	valid_1's binary_logloss: 0.414682
[1200]	valid_0's binary_logloss: 0.408184	valid_1's binary_logloss: 0.414599
[1400]	valid_0's binary_logloss: 0.406977	valid_1's binary_logloss: 0.414551
[1600]	valid_0's binary_logloss: 0.405827	valid_1's binary_logloss: 0.414542
[1800]	valid_0's binary_logloss: 0.404703	valid_1's binary_logloss: 0.414515
[2000]	valid_0's binary_logloss: 0.403614	valid_1's binary_logloss: 0.414489
Early stopping, best iteration is:
[1987]	valid_0's binary_logloss: 0.403684	valid_1's binary_logloss: 0.414485
[0.41448537100860966]
cv: 0.41448537100860966
test mean: 0.

[1800]	valid_0's binary_logloss: 0.404726	valid_1's binary_logloss: 0.414035
[2000]	valid_0's binary_logloss: 0.40361	valid_1's binary_logloss: 0.414015
[2200]	valid_0's binary_logloss: 0.402517	valid_1's binary_logloss: 0.414013
Early stopping, best iteration is:
[2143]	valid_0's binary_logloss: 0.402824	valid_1's binary_logloss: 0.414005
[0.41428299076783853, 0.41425358025381603, 0.4140054562022653]
cv: 0.4141806757413066
test mean: 0.215225213503568
Training until validation scores don't improve for 200 rounds.
[200]	valid_0's binary_logloss: 0.415746	valid_1's binary_logloss: 0.416857
[400]	valid_0's binary_logloss: 0.413566	valid_1's binary_logloss: 0.415737
[600]	valid_0's binary_logloss: 0.411994	valid_1's binary_logloss: 0.41536
[800]	valid_0's binary_logloss: 0.410592	valid_1's binary_logloss: 0.415153
[1000]	valid_0's binary_logloss: 0.409289	valid_1's binary_logloss: 0.415039
[1200]	valid_0's binary_logloss: 0.408044	valid_1's binary_logloss: 0.414945
[1400]	valid_0's binary

In [16]:
aaa

NameError: name 'aaa' is not defined

In [ ]:
gc.collect()
df_fea, retain_index, ifdrop = test_on_sample(train_csr_f,train_y,lgb_model,n_fold = 5,seed=19,first=2500)

In [ ]:
gc.collect()
df_fea, retain_index, ifdrop = test_on_sample(train_csr_f,train_y,lgb_model,n_fold = 5,seed=19,first=2500)

In [ ]:
aa

In [ ]:
df_fea, retain_index, ifdrop = test_on_sample(train_csr_f,train_y,lgb_model,n_fold = 5,seed=19,first=2500)

In [ ]:
gc.collect()
df_fea, retain_index, ifdrop = test_on_sample(train_csr_f,train_y,lgb_model,n_fold = 5,seed=19,first=2200)

In [ ]:
first=2400
split_index = df_fea.sort_values('split',ascending=False)['split'].index.values[:first]
gain_index = df_fea.sort_values('gain',ascending=False)['gain'].index.values[:first]
retain_index = list(set(split_index).intersection(set(gain_index)))
print(len(retain_index))
verify_drop = verify_on_sample(train_csr_f,train_y,lgb_model,retain_index,n_fold = 5,seed=19,cv_list=[0.4144021068865735])

In [ ]:
df_fea, retain_index, ifdrop = test_on_sample(train_csr,train_y,lgb_model,n_fold = 5,seed=19,first=1500)

In [ ]:
first=2500
split_index = df_fea.sort_values('split',ascending=False)['split'].index.values[:first]
gain_index = df_fea.sort_values('gain',ascending=False)['gain'].index.values[:first]
retain_index = list(set(split_index).intersection(set(gain_index)))
print(len(retain_index))
verify_drop = verify_on_sample(train_csr,train_y,lgb_model,retain_index,n_fold = 5,seed=19,cv_list=[0.41467523398442835])

In [ ]:
first=3500
split_index = df_fea.sort_values('split',ascending=False)['split'].index.values[:first]
gain_index = df_fea.sort_values('gain',ascending=False)['gain'].index.values[:first]
retain_index = list(set(split_index).intersection(set(gain_index)))
print(len(retain_index))
verify_drop = verify_on_sample(train_csr,train_y,lgb_model,retain_index,n_fold = 5,seed=19,cv_list=[0.41467523398442835])

In [ ]:
first=4500
split_index = df_fea.sort_values('split',ascending=False)['split'].index.values[:first]
gain_index = df_fea.sort_values('gain',ascending=False)['gain'].index.values[:first]
retain_index = list(set(split_index).intersection(set(gain_index)))
print(len(retain_index))
verify_drop = verify_on_sample(train_csr,train_y,lgb_model,retain_index,n_fold = 5,seed=19,cv_list=[0.41467523398442835])

In [ ]:
first=1000
split_index = df_fea.sort_values('split',ascending=False)['split'].index.values[:first]
gain_index = df_fea.sort_values('gain',ascending=False)['gain'].index.values[:first]
retain_index = list(set(split_index).intersection(set(gain_index)))
print(len(retain_index))
verify_drop = verify_on_sample(train_csr,train_y,lgb_model,retain_index,n_fold = 5,seed=19,cv_list=[0.41467523398442835])

In [ ]:
first=5000
split_index = df_fea.sort_values('split',ascending=False)['split'].index.values[:first]
gain_index = df_fea.sort_values('gain',ascending=False)['gain'].index.values[:first]
retain_index = list(set(split_index).intersection(set(gain_index)))
print(len(retain_index))
verify_drop = verify_on_sample(train_csr,train_y,lgb_model,retain_index,n_fold = 5,seed=19,cv_list=[0.41467523398442835])

In [22]:
first=3500
split_index = df_fea.sort_values('split',ascending=False)['split'].index.values[:first]
gain_index = df_fea.sort_values('gain',ascending=False)['gain'].index.values[:first]
retain_index = list(set(split_index).intersection(set(gain_index)))

train_csr_save=train_csr_f[:,retain_index]
predict_csr_save=predict_csr_f[:,retain_index]

save_csr(train_csr_save,'../../data/features/xgb/train_csr_{}'.format(first))
save_csr(predict_csr_save,'../../data/features/xgb/predict_csr_{}'.format(first))

data save
indices save
indptr save
save done!
data save
indices save
indptr save
save done!


In [17]:
df_fea.to_csv('../../data/features/xgb/fea_importance.csv',index=False)